# Create the ratings matrix

In [3]:
import pandas as pd
import numpy as np
from glob import glob

users_dir = "../dataset/utility/users"
items_dir = "../dataset/utility/items"

In [4]:
items_df = pd.read_csv(f"{items_dir}/itemset.csv", low_memory=False)
items_df.rename(columns={"Unnamed: 0": "ASIN"}, inplace=True)
items_df.set_index("ASIN", inplace=True)
display(items_df)

,Home & Kitchen,Bedding,Comforters & Sets,Comforter Sets,Kids' Bedding,Baby Products,Nursery,Toddler Bedding,Bedding Sets,Blankets & Throws,...,Torches,Pest Control,Bug Zappers,Bistro Sets,Outdoor Curtains,Patio Furniture Covers,Furniture Set Covers,Figurine Lights,Storage Benches,Boot & Shoe Boxes
ASIN,,,,,,,,,,,,,,,,,,,,,
B0CTM6P5TW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0CTH3XT3D,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0CT2CC1YY,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0CSSRBG48,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0CS4CP75C,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B07QXK9WG5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0CHXTR17D,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0C9MZWQ1D,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
utility_csvs = glob(f"{users_dir}/utility_*")
users_df_list = []
for util in utility_csvs:
    users_df_list.append(pd.read_csv(util))

# Try all algorithms

## Content-based

### Unary Aggregation

In [ ]:
def compute_user_profile_agg_unary(df_utility, df_item_profiles, user):
    """
    Return user profile with unarized ratings
    """
    utility_user_arr = df_utility.loc[user].to_numpy()
    user_mean = np.nanmean(utility_user_arr)
    unarized_ratings = []
    for i in range(len(utility_user_arr)):
        if ~np.isnan(utility_user_arr[i]):
            if utility_user_arr[i] < user_mean:
                unarized_ratings.append(0)
            else:
                unarized_ratings.append(1)
        else:
            unarized_ratings.append(np.nan)
    unarized_ratings = np.array(unarized_ratings)
    ones = np.count_nonzero(unarized_ratings == 1)
    indices = list(np.nonzero(~np.isnan(unarized_ratings))[0])
    df_item_profiles_relevant = df_item_profiles.values[indices]
    return np.dot(
        np.array(unarized_ratings)[indices],
        df_item_profiles_relevant)/np.sum(ones)

## Collaborative and Matrix Factorization

From Surprise

In [ ]:
from surprise import AlgoBase, Dataset, accuracy, Reader
from surprise.model_selection import cross_validate, train_test_split
from surprise import PredictionImpossible
from surprise.prediction_algorithms.knns import KNNBasic, KNNWithMeans, KNNWithZScore
from surprise.prediction_algorithms.matrix_factorization import SVD, SVDpp, NMF
from surprise.prediction_algorithms.co_clustering import CoClustering
import numpy as np

In [ ]:
reader = Reader(rating_scale=(0,5))
melted_user_df = users_df.reset_index().melt(
    'reviewerID', var_name='ASIN', value_name='rating').dropna()
dataset = Dataset.load_from_df(melted_user_df, reader)
algos = [KNNBasic(), KNNWithMeans(), KNNWithZScore(), SVD(), SVDpp(), NMF(), CoClustering()]
for algo in algos:
    cross_validate(algo, dataset, verbose=True)

In [ ]:
# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin("ml-100k")

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=0.25)

# We'll use the famous SVD algorithm.
algo = MyOwnAlgorithm()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)